# Generate CSV for CLAM dataset splitting
## Input:
1. Sample clustering results (cluster label)
2. CSV auto-generated by CLAM segmentation (slide id)

## Ouput:
1. CSV for CLAM dataset splitting (case id, slide id, cluster)

In [2]:
import os
import pandas as pd

In [3]:
# Only keep core samples
core_sample = False # only for 340
highvsrest = True # Median + Low
lowvsrest = False # High + Median
amount = 349 # 354

In [3]:
# # 4 versions of labels
# root = "/media/visiopharm5/WDGold/deeplearning/MIL/CLAM"
# if core_sample:
#     path_label = "/media/visiopharm5/WDGold/deeplearning/Hepatocarcinomes/TCGA/clust_res_final"
# else:
#     path_label = "/media/visiopharm5/WDGold/deeplearning/Hepatocarcinomes/TCGA/heatmap"

# ffname = "results/process_list_edited (copy).csv" # 20x + 40x
# # merge = "outer"

# flabel = "sample_clusters_filtered-4_zscore_hc_ward.D2_euclidean_3_reorder.csv" # v1
# output = "dataset_csv/tcga_hcc_354_v1"

# # flabel = "sample_clusters_sum_filter-4.csv" # v2
# # output = "dataset_csv/tcga_hcc_v2"

# # flabel = "sample_clusters_filtered-Sangro_zscore_hc_ward.D2_euclidean_3_reorder.csv" # v3
# # output = "dataset_csv/tcga_hcc_354_v3"

# # flabel = "sample_clusters_full_median_cnmf_ordered_hc_ward.D2_euclidean_3.csv" # v4
# # output = "dataset_csv/tcga_hcc_354_v4"

# if highvsrest:
#     output = output + "_highvsrest"

# if core_sample:
#     output = output + "_core.csv"
# else:
#     output = output + ".csv"

In [34]:
# Gene signatures in Sangro's paper
root = "/media/visiopharm5/WDGold/deeplearning/MIL/CLAM"
if core_sample:
    path_label = "/media/visiopharm5/WDGold/deeplearning/Hepatocarcinomes/TCGA/clust_res_final/tcga_"
else:
#     path_label = "/media/visiopharm5/WDGold/deeplearning/Hepatocarcinomes/TCGA/validation_340/"
    path_label = "/media/visiopharm5/WDGold/deeplearning/Hepatocarcinomes/TCGA/validation_336_filter_before/"

if amount == 354:
    ffname = "results/process_list_edited (copy).csv" # 20x + 40x
elif amount == 349:
    ffname = "results/process_list_edited (copy 349).csv" # 20x + 40x
else:
    raise ValueError
# merge = "outer"

gene_signature = "6G_Interferon_Gamma" # Inflammatory, Gajewski_13G_Inflammatory, 6G_Interferon_Gamma, 
                                        # Interferon_Gamma_Biology, T-cell_Exhaustion, Ribas_10G_Interferon_Gamma

flabel = "sample_clusters_"+ gene_signature +"_zscore_hc_ward.D2_euclidean_3_reorder.csv"
output = "dataset_csv/tcga_hcc_"+str(amount)+"_" + gene_signature


if highvsrest:
    output = output + "_highvsrest"
elif lowvsrest:
    output = output + "_lowvsrest"

if core_sample:
    output = output + "_core.csv"
else:
    output = output + ".csv"

In [35]:
# Load clustering results
labels = pd.read_csv(path_label+flabel)
# Load CSV auto-generated by CLAM segmentation -- for full slide name
df = pd.read_csv(os.path.join(root, ffname))

In [36]:
# Data vis
print(labels.shape)
display(labels.head(5))

print(df.shape)
display(df.head(5))

(336, 2)


,Sample,Cluster
0,AADN.01A,Cluster High
1,AACC.01A,Cluster High
2,AACB.01A,Cluster High
3,A5RG.01A,Cluster High
4,A95S.01A,Cluster High


(349, 17)


,slide_id,process,status,seg_level,sthresh,mthresh,close,use_otsu,a_t,a_h,max_n_holes,vis_level,line_thickness,white_thresh,black_thresh,use_padding,contour_fn
0,TCGA-2V-A95S-01Z-00-DX1.9DFA6FC7-87A6-4C14-9A3...,0,processed,2,15,7,4,True,20,0.1,8,2,50,8,40,True,four_pt
1,TCGA-2Y-A9GS-01Z-00-DX1.75C5CEF1-6C8B-42FF-82F...,0,processed,3,15,7,4,True,20,0.1,8,3,200,8,40,True,four_pt
2,TCGA-2Y-A9GT-01Z-00-DX1.30666775-3556-4DFE-A5E...,0,processed,3,15,7,1,False,20,0.1,50,3,200,5,40,True,four_pt
3,TCGA-2Y-A9GU-01Z-00-DX1.700CBBD7-9F58-470D-A85...,0,processed,3,8,7,2,True,100,2.0,8,3,200,8,40,True,four_pt
4,TCGA-2Y-A9GV-01Z-00-DX1.524FA6DD-7C84-425C-A2B...,0,processed,3,15,7,4,False,100,2.0,50,3,200,5,40,True,four_pt


In [37]:
# Re-format sample name: remove "X" before all numeric names and replace "." with "-"
for i in range(labels.shape[0]):
    labels.iloc[i,0] = labels.iloc[i,0][-8:-1].replace(".", "-")
print(labels.shape)
display(labels.head(5))

(336, 2)


,Sample,Cluster
0,AADN-01,Cluster High
1,AACC-01,Cluster High
2,AACB-01,Cluster High
3,A5RG-01,Cluster High
4,A95S-01,Cluster High


In [38]:
if highvsrest:
    labels.replace("Cluster Median", "Cluster Median + Low", inplace=True)
    labels.replace("Cluster Low", "Cluster Median + Low", inplace=True)
elif lowvsrest:
    labels.replace("Cluster High", "Cluster High + Median", inplace=True)
    labels.replace("Cluster Median", "Cluster High + Median", inplace=True)
display(labels.head(5))

,Sample,Cluster
0,AADN-01,Cluster High
1,AACC-01,Cluster High
2,AACB-01,Cluster High
3,A5RG-01,Cluster High
4,A95S-01,Cluster High


In [39]:
if core_sample:
    for i in range(labels.shape[0]): # redefine "core" sample
        if labels.iloc[i, 1] == labels.iloc[i, 2]:
            labels.iat[i, 4] = True
    
    labels = labels.drop(labels[~labels.core_sample].index).drop(columns=["neighbor", "sil_width", "core_sample"])
    print(labels.shape)
    display(labels.head(5))

In [40]:
# Build a new df
df1 = df.filter(items=["slide_id"], axis="columns")
df1["case_id"] = ""
df1["Sample"] = ""
for i in range(df.shape[0]):
    df1.iloc[i,0] = df.iloc[i,0][:-4] # remove ".svs" at the end
    df1.iloc[i,1] = df.iloc[i,0][0:12] # case id
    df1.iloc[i,2] = df.iloc[i,0][8:15] # sample id which used to match the label df
print(df1.shape)
display(df1.head(5))

(349, 3)


,slide_id,case_id,Sample
0,TCGA-2V-A95S-01Z-00-DX1.9DFA6FC7-87A6-4C14-9A3...,TCGA-2V-A95S,A95S-01
1,TCGA-2Y-A9GS-01Z-00-DX1.75C5CEF1-6C8B-42FF-82F...,TCGA-2Y-A9GS,A9GS-01
2,TCGA-2Y-A9GT-01Z-00-DX1.30666775-3556-4DFE-A5E...,TCGA-2Y-A9GT,A9GT-01
3,TCGA-2Y-A9GU-01Z-00-DX1.700CBBD7-9F58-470D-A85...,TCGA-2Y-A9GU,A9GU-01
4,TCGA-2Y-A9GV-01Z-00-DX1.524FA6DD-7C84-425C-A2B...,TCGA-2Y-A9GV,A9GV-01


In [41]:
# Matching the two df on sample id
results = pd.merge(labels, df1, on="Sample", how="inner") # keep the matched duplicates (any case using merge 
                                                          # function) and preserve the left order
print(results.shape)
display(results.head(5))

(349, 4)


,Sample,Cluster,slide_id,case_id
0,AADN-01,Cluster High,TCGA-DD-AADN-01Z-00-DX1.0C031278-22F9-40DE-9CF...,TCGA-DD-AADN
1,AACC-01,Cluster High,TCGA-DD-AACC-01Z-00-DX1.5B3061BB-6716-4253-851...,TCGA-DD-AACC
2,AACB-01,Cluster High,TCGA-DD-AACB-01Z-00-DX1.D4A7846E-8B38-4FBA-BAB...,TCGA-DD-AACB
3,A5RG-01,Cluster High,TCGA-K7-A5RG-01Z-00-DX1.D8137CB8-34DC-41C0-AE4...,TCGA-K7-A5RG
4,A95S-01,Cluster High,TCGA-2V-A95S-01Z-00-DX1.9DFA6FC7-87A6-4C14-9A3...,TCGA-2V-A95S


In [42]:
# Reconstruct the results
results = results.filter(items=["case_id", "slide_id", "Cluster"], axis="columns")
results.rename(columns={"Cluster":"cluster"}, inplace=True)
print(results.shape)
display(results.head(5))

(349, 3)


,case_id,slide_id,cluster
0,TCGA-DD-AADN,TCGA-DD-AADN-01Z-00-DX1.0C031278-22F9-40DE-9CF...,Cluster High
1,TCGA-DD-AACC,TCGA-DD-AACC-01Z-00-DX1.5B3061BB-6716-4253-851...,Cluster High
2,TCGA-DD-AACB,TCGA-DD-AACB-01Z-00-DX1.D4A7846E-8B38-4FBA-BAB...,Cluster High
3,TCGA-K7-A5RG,TCGA-K7-A5RG-01Z-00-DX1.D8137CB8-34DC-41C0-AE4...,Cluster High
4,TCGA-2V-A95S,TCGA-2V-A95S-01Z-00-DX1.9DFA6FC7-87A6-4C14-9A3...,Cluster High


In [43]:
# Export
results.to_csv(os.path.join(root, output))